In [ ]:
##### General ToDo list:

# For select_data() function, add "STAGE/ROUTE" selection
#     Add an ooption to view screening 'STAGE/ROUTE' only as well

#//ToDo: write a function to fliter out only singificant breaches/ significant number of cases pe trust etc

# Select_data() is now redundant with filter_data()

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go


In [2]:
from data_wrangling import get_national
from cancer_plotting import plot_stacked_referrals

In [3]:
# code example being used - note will need to run the national_data code first 
national_data_link = r'https://www.england.nhs.uk/statistics/wp-content/' \
    + 'uploads/sites/2/2023/12/' \
    + 'CWT-CRS-National-Time-Series-Oct-2009-Oct-2023-with-Revisions.xlsx'

national_data = get_national(national_data_link)
labels = ['Within Standard_28', 'Outside Standard_28', 'Within Standard_31', 'Outside Standard_31'] 
data_to_plot=['Month','Within Standard_28', 'Outside Standard_28', 'Within Standard_31', 'Outside Standard_31']
fig, ax = plot_stacked(national_data[data_to_plot],labels,"Patients")

NameError: name 'plot_stacked' is not defined

In [4]:
from data_wrangling import get_provider_data, nhs_code_link, help_with, read_icb_sicb_coding
from data_wrangling import select_months, select_org, select_cancer, select_standard
from data_wrangling import select_data
from data_wrangling import proportion_breaches

from data_wrangling import filter_data

from cancer_plotting import breaches_animated_plot


In [5]:
data = get_provider_data()

In [42]:
def select_cancer(df, cancer_type, strict=False):
    """

    Parameters
    ----------
    - df : Dataframe
        Dataframe which requires filtering
    - cancer_type_list : List 
        List of cancer types you wish to include in data frame.
        e.g. for all breast:
            cancer_type_list = [Unsuspected_breast_ca', 'Suspected_breast_ca']

    Raises
    ------
    ValueError
        If the cancer type list contains a cancer type not in the dataframe.

    Returns
    -------
    df : Dataframe
        Dataframe containing only cancer types in the cancer_type_list

    """
    
    if isinstance(cancer_type, str):
        if not df['cancer_type'].eq(cancer_type).any():
            raise ValueError(
                 f'Cancer type "{cancer_type}" are not in the dataframe')
        else:
            df = df.loc[df.cancer_type ==cancer_type]
            return df
        

    elif isinstance(cancer_type, list):
        # check to see if each string in the cancer type list is in the dataframe.
        for can in cancer_type:
            if not df['cancer_type'].eq(can).any():
                if strict:
                    raise ValueError(
                        f'Cancer type "{cancer_type}" are not in the dataframe')
                    break
                if not strict:
                    print(f'Error ocurred with value {can}, removing it and continuing')
                    cancer_type.remove(can)
                    continue
            else:
                continue
                
            # filters the dataframe based on the cancer type list
        df = df[df['cancer_type'].isin(cancer_type)]
        return df

In [51]:
x = select_cancer(data, cancer_type=['whatevs','Unsuspected_breast_ca', 'All_Cancers'])

Error ocurred with value whatevs, removing it and continuing


In [54]:
x.cancer_type.unique()

['Unsuspected_breast_ca', 'All_Cancers']
Categories (28, object): ['All_Cancers', 'Breast', 'Unsuspected_breast_ca', 'Gynecological', ..., 'Suspected_upper_GI_ca', 'Suspected_urological_ca', 'Upper_GI', 'Urological']

In [26]:
select_cancer('')

TypeError: select_cancer() missing 1 required positional argument: 'cancer_type'

In [ ]:
filter_data(data, ['radiotherapy','not_applicable_FDS'], [])

In [ ]:
#data = proportion_breaches((data))

In [ ]:
select_months(data, end_date='2023-10-01')

In [ ]:
help(filter_data)

In [ ]:
filter_data(df=data, filters={'start_month':'05-2022',
                              'end_month':'05-2022',
                              'standard':'FDS'})

In [ ]:
help(select_standard)

In [ ]:
isinstance(np.empty(shape=2), list)

In [ ]:
help_with('standards')

In [ ]:
link_data = nhs_code_link()
# overlap = list(set(data['ORG CODE']) & set(link_data['ORG CODE']))
# unmatched = data[~data['ORG CODE'].astype(str).isin(link_data['ORG CODE'].astype(str))]

In [ ]:
#plt.figure(figsize=[10,10])
sns.stripplot(x='PERIOD', y = 'BREACHES', data=select_org(df=data,org_str='R1K'), hue = 'CANCER_TYPE', legend=False)

In [ ]:
sns.lineplot(x='PERIOD', y='BREACHES', data=select_org(data,'R0A').groupby(by='PERIOD').sum())

In [ ]:
data

In [ ]:
# Need to add informative title displaying type of cancer and full name of organisation + national standard on the plot
# Need to handle if some orgs do not have certain cancers/standards

trial = select_data(data,[('cancer',3), ('org', 'r0a'), ('standard', 'FDS')])
trial = trial.loc[trial['STAGE/ROUTE'] =='NATIONAL SCREENING PROGRAMME']
breaches_animated_plot(trial,[])


In [ ]:
def create_lookup_dict_icb():
    
    icb_codes = read_icb_sicb_coding()
    nhs_link = nhs_code_link()
    
    #Creates dictionary of ICB 3-digit code to ICB Name
    icb_code_to_names = dict(zip(icb_codes['ICB22CDH'], icb_codes['ICB22NM']))
    
    #Creates dictionary of NHS Trust Org code to Higher Level Health Geography
    org_to_hlhg = dict(zip(nhs_link['ORG_CODE'], nhs_link['Higher Level Health Geography']))
    
    return icb_code_to_names, org_to_hlhg
    

In [ ]:
######## Below is work in progress on creating a map with statistics

In [ ]:
def select_to_plot(data, filter_to_apply=[], gdf = gdf):
    
    filtered_df = select_data(data, filter_to_apply)
    
    
    icb_code_to_names, org_to_hlhg = create_lookup_dict_icb()
    
    
    filtered_df['hlhg'] = filtered_df['ORG_CODE'].map(org_to_hlhg)
    filtered_df['ICB23NM'] = filtered_df['hhlg'].map(icb_code_to_names)
    
    labels_for_plotting = {'cancer_type':filtered_df.CANCER_TYPE.unique(),
                           'period':filtered_df.PERIOD.unique(),
                           'standard':filtered_df.STANDARD.unique()
                          }
    
    icb_breaches = round(filtered_df.groupby('ICB23NM').BREACHES.sum()
                         / filtered_df.groupby('ICB23NM').TOTAL.sum(), 2
                        ).astype(float)
    
    
    icb_breaches.rename('proportion_breaches', inplace=True)
    
    merged_gdf = pd.merge(gdf, icb_breaches, left_on='ICB23NM', right_index=True)
    
    return merged_gdf, labels_for_plotting
    
    
    
    

In [ ]:
help(select_to_plot)

In [ ]:
labels = select_to_plot(data, [('cancer',10), ('standard', 'FDS')])[1]
plotting_material = select_to_plot(data, [('cancer',10), ('standard', 'FDS')])[0]

In [ ]:
labels

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
plotting_material.plot(column = 'proportion_breaches',
                           ax=ax, legend=True,
                           legend_kwds={'label': "Proportion Breaches", 'orientation': "vertical"})

plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)


In [ ]:
icb_codes = read_icb_sicb_coding()
nhs_link = nhs_code_link()

icb_code_to_names = dict(zip(icb_codes['ICB22CDH'], icb_codes['ICB22NM']))
org_to_icb = dict(zip(nhs_link['ORG_CODE'], nhs_link['Higher Level Health Geography']))

data['hhlg'] = data['ORG_CODE'].map(org_to_icb)
data['ICB23NM'] = data['hhlg'].map(icb_code_to_names)

In [ ]:
icb_breaches = round(data.groupby('ICB23NM').BREACHES.sum() / data.groupby('ICB23NM').TOTAL.sum(), 2).astype(float)
icb_breaches.rename('proportion_breaches', inplace=True)

In [ ]:
import geopandas as gpd
path_to_shapefile = 'data/ons_shapefile/Integrated_Care_Boards_April_2023_EN_BFC_1659257819249669363/'

In [ ]:
gdf = gpd.read_file(path_to_shapefile)

In [ ]:
gdf.head()

In [ ]:
merged_gdf = pd.merge(gdf, icb_breaches, left_on='ICB23NM', right_index=True)
merged_gdf.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
merged_gdf.plot(column='LAT', ax=ax, legend=True,
                legend_kwds={'label': "Proportion Breaches", 'orientation': "vertical"})

plt.show()